In [2]:
import cv2, numpy as np

In [ ]:
## orb로 특징점 추출 >> 호모그래피로 변환행렬 생성 >> 원근변환

src = cv2.imread("image/apple_book.jpg", cv2.IMREAD_)
target = cv2.imread('image/apple.jpg', 0)

## orb 객체 생성
orb = cv2.ORB_create(
    nfeatures = 40000,
)

## 키포인트, 디스크립터 생성
kp1, des1 = orb.detectAndCompute(src, None)
kp2, des2 = orb.detectAndCompute(target, None)

## 비교후 정렬
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x: x.distance)

## src 포인터, dst 포인터 생성
src_pts = np.float32([kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
target_pts = np.float32([kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)

## 호모그래피로 변환행렬 추출
mtrx, mask = cv2.findHomography(src_pts, target_pts, cv2.RANSAC, 5.0)

h, w = target.shape

pts = np.float32([[[0,0]],[[0, h]],[[w,h]],[[w,0]]])

## 원근변환
dst = cv2.perspectiveTransform(pts,mtrx)
matrix = cv2.getPerspectiveTransform(dst,pts)
warp = cv2.warpPerspective(src, matrix, (w,h))

## mask를 지정 해 좋은 매칭결과만 그린다
matchesMask = mask.ravel().tolist()
res = cv2.drawMatches(src,kp1,target,kp2,matches,None,
                      matchesMask = matchesMask,
                      flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)





cv2.imshow('res', res)
cv2.imshow("warp", warp)
cv2.waitKey()
cv2.destroyAllWindows()

In [32]:
## orb,sift,surf 로 ir 특징점 추출 해 보기

img1 = cv2.imread('image/ir test/ir6.jpg',0)
img2 = cv2.imread('image/ir test/rgb6.jpg',0)
img2 = cv2.resize(img2, dsize = (0,0), fx = 0.5, fy = 0.5)


img1 = cv2.Canny(img1,100,250)
img2 = cv2.Canny(img2,75,200)



##detector = cv2.xfeatures2d.SURF_create()
detector = cv2.xfeatures2d.SIFT_create()
##detector = cv2.ORB_create()


kp1, desc1 = detector.detectAndCompute(img1, None)
kp2, desc2 = detector.detectAndCompute(img2, None)

matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)
matches = matcher.match(desc1, desc2)

matches = sorted(matches, key=lambda x:x.distance)

## 상위 특징점만 그린다
for i in matches[:50]:
    idx_des1 = i.queryIdx
    x1, y1, =  kp1[idx_des1].pt
    
    idx_des2 = i.trainIdx
    x2, y2, =  kp2[idx_des2].pt
    
    cv2.circle(img1, (int(x1), int(y1)), 3, (0, 0, 0), 3)
    cv2.circle(img2, (int(x2), int(y2)), 3, (255, 0, 0), 3)


res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None,flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('res1', res1)
cv2.imshow('image1', img1)
cv2.imshow('image2 ', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [29]:
## 매칭

img1 = cv2.imread('image/ir test/ir2-1.jpg',0)
img2 = cv2.imread('image/ir test/rgb2-1.jpg',0)
img2 = cv2.resize(img2, dsize = (0,0), fx = 0.5, fy = 0.5)


 

detector = cv2.xfeatures2d.SIFT_create(contrastThreshold = 0.2)


kp1, desc1 = detector.detectAndCompute(img1, None)
kp2, desc2 = detector.detectAndCompute(img2, None)

matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)
matches = matcher.match(desc1, desc2)

matches = sorted(matches, key=lambda x:x.distance)


src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)



mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
h,w = img1.shape
pts = np.float32([ [[0,0]],[[0,h]],[[w,h]],[[w,0]] ])
dst = cv2.perspectiveTransform(pts,mtrx)

matrix = cv2.getPerspectiveTransform(dst,pts)
img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
warp = cv2.warpPerspective(img2, matrix, (w , h))




matchesMask = mask.ravel().tolist()
res2 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                    matchesMask = matchesMask,
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)



# 결과 출력                    
cv2.imshow('warp', warp)
cv2.imshow('res2 ', res2)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
img1 = cv2.imread('image/ir test/ir2-1.jpg',0)
img2 = cv2.imread('image/ir test/rgb2.jpg',0)
## img1 = cv2.resize(img1, dsize = (0,0), fx = 1.5, fy = 1.5)

img1 = cv2.Canny(img1,100,250)
img2 = cv2.Canny(img2,75,200)
detector = cv2.xfeatures2d.SIFT_create()


kp1, desc1 = detector.detectAndCompute(img1, None)
kp2, desc2 = detector.detectAndCompute(img2, None)

matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)
matches = matcher.match(desc1, desc2)

matches = sorted(matches, key=lambda x:x.distance)


src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)



mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
h,w = img1.shape
pts = np.float32([ [[0,0]],[[0,h]],[[w,h]],[[w,0]] ])
dst = cv2.perspectiveTransform(pts,mtrx)

matrix = cv2.getPerspectiveTransform(dst,pts)
img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
warp = cv2.warpPerspective(img2, matrix, (w , h))




matchesMask = mask.ravel().tolist()
res2 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                    matchesMask = matchesMask,
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('warp', warp)
cv2.imshow('res2 ', res2)
cv2.imshow('canny1', img1)
cv2.imshow('canny2', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
print(img2.tolist())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

